In [1]:
from time import time
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from keras import regularizers
import tensorflow as tf
import keras
import pandas as pd
from sklearn import preprocessing

In [2]:
X_all=np.load("train/train_gray.npy")

In [3]:
X_all = X_all.astype('float16')
X_all = X_all/255

In [10]:
X_all=X_all.reshape(X_all.shape[0],99,161)

In [12]:
noise=np.load("_background_noise_/noise_grey.npy")

In [13]:
noise = noise.astype('float16')
noise = noise/255

In [15]:
noise=noise.reshape(noise.shape[0],99,161)

In [16]:
noise.shape

(406, 99, 161)

# Model

In [26]:
width_factor=1
weight_decay = 1e-4
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32*width_factor, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(99,161,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32*width_factor, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(64*width_factor, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64*width_factor, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64*width_factor, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(30, activation='softmax'))

In [28]:
width_factor=1
weight_decay = 1e-4
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(16))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64*width_factor, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(30, activation='softmax'))

In [29]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

In [30]:
#train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
#train_datagen.fit(x_train)

In [31]:
#start=time()
#history=model.fit(train_datagen.flow(x_train, y_train, batch_size=64, seed=123),
#          steps_per_epoch=len(x_train) / 64, epochs=2, validation_data=(valid_norm, y_val))
#end=time()

In [ ]:
start=time()
history=model.fit(x=x_train,y=y_train,batch_size=64,
          steps_per_epoch=len(x_train) / 64, epochs=2, validation_data=(valid_norm, y_val))
end=time()

In [ ]:
print((end-start)//60)

acc = model.evaluate(test_norm, y_test)

acc

In [ ]:
def plot(losses, accuracies, subplot_title,y_type):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,4),sharey=True)
  
    ax1.plot(losses)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel(y_type)
    
    ax1.set_title(subplot_title[0])
  
    ax2.plot(accuracies)
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel(y_type)
    ax2.set_title(subplot_title[1])
    
    plt.show()

# Training
plot(history.history['loss'], 
     history.history['val_loss'], 
     subplot_title=['Training Loss', 'Validation Loss'],y_type="Loss"
    )

# Validation
plot(history.history['acc'], 
     history.history['val_acc'], 
     subplot_title=['Training Accuracy', 'Validation Accuracy'],y_type="Accuracy"
    )

In [ ]:
# Save Results

model_type="conv1"

model.save("model_"+model_type)

import shutil
shutil.make_archive("model_"+model_type, 'zip', "model_"+model_type)


import pandas as pd
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history_'+model_type+'.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
#!cd model && 7z e my_model.zip

imported = tf.keras.models.load_model("./model",compile=False)

import pandas as pd
history=pd.read_csv("history_block4_width1_reg_full.csv")

history

# Training
plot(history['loss'], 
     history['val_loss'], 
     subplot_title=['Training Loss', 'Validation Loss'],y_type="Loss"
    )

# Validation
plot(history['acc'], 
     history['val_acc'], 
     subplot_title=['Training Accuracy', 'Validation Accuracy'],y_type="Accuracy"
    )